# Imports

In [1]:
%%capture
%load_ext autoreload
from helper import *
%autoreload
%aimport helper

2023-04-25 00:47:25.455924: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 00:47:25.511529: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 00:47:25.512241: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 00:47:26.491404: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Data-transformation for mid term

In [2]:
# setting up features and target
cols_std = ['ttf_price', 'coal_price', 'co2_price','DK_1_imports_mean4',
       'SE_4_imports_mean4', 'DK_1_exports_mean4', 'SE_4_exports_mean4',
       'Actual_Load_mean4', 'Solar_[MW]_mean4', 'DE_LU_AT_imports_mean4',
       'DE_LU_AT_exports_mean4', 'Wind Total_mean4'] 
features = cols_std + ['Quarter', 'Month', 'Year', 'Weekday', 'Hour', 'Week', 'business']
target = 'Day_Ahead_Prices'

In [4]:
# setting up splits
train_start = pd.to_datetime(['2018-01-01', '2021-01-01', '2021-07-01'], format = '%Y-%m-%d')
k_folds = len(train_start)


df = pd.read_csv('dataset_management\data\clean\mean_lagged.csv', parse_dates=['Timestamp'])
df=df[:-288]

df.columns

Index(['Timestamp', 'Day_Ahead_price', 'ttf_price', 'coal_price', 'co2_price',
       'Hour', 'Weekday', 'business', 'DK_1_imports_mean4',
       'SE_4_imports_mean4', 'DK_1_exports_mean4', 'SE_4_exports_mean4',
       'Actual_Load_mean4', 'Solar_[MW]_mean4', 'DE_LU_AT_imports_mean4',
       'DE_LU_AT_exports_mean4', 'Wind Total_mean4'],
      dtype='object')

# Models

## Split - Method = 0
The approach is to train and test in consecutive/sequential blocks, devoting 85% to training and 15% to test.

In [ ]:
split_method = 0
df_lagged=df_lagged.drop(['Day_Ahead_Prices-lag-168'], axis = 1)


### Baseline

In [ ]:
lr_pred_0, lr_model_0 = run_model('lr', df_lagged1, k_folds, split_method,
                                         train_start, features, target, cols_std)

In [ ]:
lr_model_0

### Moving average

In [ ]:
df_ma_0 = run_model('ma', df_lagged, k_folds, split_method,
                                         train_start, features, target, cols_std, pred_window = 24*1)

df_ma_0.dropna(how = 'any', axis = 0, inplace = True)
df_ma_0.drop(df_ma_0[df_ma_0[target] == 0].index, inplace = True)

In [ ]:
ytrue = np.array(df_ma_0[target]).ravel()
ypred = np.array(df_ma_0['pred']).ravel()

In [ ]:
model_evaluation(ytrue, ypred)

### Random Forest

In [ ]:
rf_pred_0, rf_models_0 = run_model('rf', df_lagged, k_folds, split_method, train_start, features, target, cols_std)

### XGBoost

In [ ]:
xgb_pred_0, xgb_models_0 = run_model('xgb', df_lagged, k_folds, split_method, train_start, features, target, cols_std,'short_term')

## Split - Method = 1
The approach is to split the data in increasing blocks. The size of the train set increases, whereas the size of the test remains 3 months.

In [ ]:
split_method = 1

In [ ]:
lr_pred_1, lr_model_1 = run_model('lr', df_lagged, k_folds, split_method,
                                         train_start, features, target, cols_std)

In [ ]:
rf_pred_1, rf_models_1 = run_model('rf', df_lagged, k_folds, split_method, train_start, features, target, cols_std, 'short_term')

In [ ]:
xgb_pred_1, xgb_models_1 = run_model('xgb', df_lagged, k_folds, split_method, train_start, features, target, cols_std)

#### NOTES FRANCISCO/FREDERIK 14/04<br>
correlation between ytrue and ypred, RMSE can be hard to refrence.<br>
use power transform instead of log (how?)

# Long term

In [5]:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import Ridge


r=df.copy()
r = r.set_index('Timestamp')

def build_far(x_train, y_train, x_test,steps): # same split as short term
    model_ = ForecasterAutoreg(regressor = Ridge(),lags = 336)
    model_.fit(y=y_train,exog=x_train)
    ypred = model_.predict(steps = steps,exog = x_test)
# Add datetime index to predictions
    #ypred = pd.Series(data=ypred, index=x_test.index)
    return ypred, model_